<a href="https://colab.research.google.com/github/yeop6134/2022-Basic-Track-Assignment/blob/main/BERT%EA%B8%B0%EB%B0%98TopicModeling/KoBERTopic_%EB%B6%88%EC%9A%A9%EC%96%B4%EC%A0%9C%EA%B1%B0__Okt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic[visualization]

# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
%pip install bertopic

In [ ]:
%pip install bertopic[visualization]

# 필요한 것들을 임포트

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab, Okt
from bertopic import BERTopic

# 데이터 불러오기

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/인공지능인문학"

/content/drive/MyDrive/Colab Notebooks/인공지능인문학


In [ ]:
documents = [line.strip() for line in open("discrimination.txt", encoding="utf-8").readlines()]
documents[:5]

['text',
 '남자들은 도대체 왜 그러는걸까?',
 '말하면서 침 개많이 튀길것 같아.',
 '머리가 멍청해서 못읽는거아니냐?',
 '"동유럽, 러시아 애들은 가난해도 사랑으로 결혼하는데 헬조선은 자본주의의 지옥 그 자체다"']

# Mecab과 SBERT를 이용한 Bertopic (형태소단위)

In [ ]:
# 불용어 사전 만들기
korean_stopwords_path = "./korean_stopwords.txt"

with open(korean_stopwords_path, encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        words = [word for word in word_tokens if len(word) > 1]

        result = []
        for w in words: 
         if w not in stopwords: 
            result.append(w)

        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Okt())

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=8,
                 top_n_words=10,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model.fit_transform(documents)

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
for i in range(0, 8):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('ㅋㅋ', 0.05834549753728123), ('여자', 0.04245572989420529), ('남자', 0.036039253920629635), ('사람', 0.02804401841750781), ('하고', 0.02804107291435116), ('아이돌', 0.024955287276803684), ('해서', 0.024142851888617225), ('하는', 0.022287155860461573), ('하면', 0.022247099826515637), ('이나', 0.021739641843068508)]
1 번째 토픽 : [('남자', 0.0433679066732046), ('이래서', 0.04190859352853407), ('이나', 0.03996733968262455), ('ㅋㅋ', 0.03803050796180049), ('전라도', 0.03583838140872677), ('처럼', 0.03467235867948489), ('주제', 0.030614583327082456), ('한테', 0.02993655293253226), ('여자', 0.027866828499375784), ('나라', 0.024431523434991116)]
2 번째 토픽 : [('여자', 0.20639676189193343), ('남자', 0.05968465132182846), ('."', 0.04575827138454587), ('자고로', 0.04549639649471896), ('여자애', 0.03649006603165433), ('머리', 0.03225623341473034), ('바지', 0.030673643397439086), ('치마', 0.029873178195334492), ('?"', 0.027488319905651353), ('예쁜', 0.027243722420922867)]
3 번째 토픽 : [('ㅋㅋ', 0.07168622978056767), ('ㅋㅋㅋ', 0.054947845287880416), ('여자', 0.

## 명사단위 분석

In [ ]:
Okt = Okt()

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.nouns(sent)

        words = [word for word in word_tokens if len(word) > 1]

        result = []
        for w in words: 
           if w not in stopwords: 
             result.append(w)
        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Okt)
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=8,
                 top_n_words=10,
                 calculate_probabilities=True)
topics, probs = model.fit_transform(documents)
for i in range(0, 8):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('남자', 0.06719653217193002), ('여자', 0.047038296396840866), ('사람', 0.0391297262160761), ('흑인', 0.03882928020061896), ('전라도', 0.03269816332553103), ('게이', 0.030334710545103276), ('짱깨', 0.02668202004850086), ('장애인', 0.023195770982281125), ('주제', 0.02315521047726523), ('원래', 0.021345616038800686)]
1 번째 토픽 : [('여자', 0.05488394602291754), ('남자', 0.04849758592845417), ('백인', 0.04627430944432057), ('사람', 0.036317497999914704), ('회사', 0.031195788355044204), ('아이돌', 0.029493852992297863), ('정도', 0.02661501024161324), ('보고', 0.024768020630938064), ('짱깨', 0.02371991553872405), ('여자친구', 0.023461361340016165)]
2 번째 토픽 : [('한국', 0.14678339306587906), ('조선족', 0.12967989748107728), ('인간', 0.1278983189687396), ('중국', 0.11717958875606106), ('중국인', 0.09448431275151153), ('인물', 0.08270596688949003), ('집단', 0.07996157005782038), ('서울', 0.0719613630173762), ('남성', 0.05799110692790931), ('사람', 0.0525785445636349)]
3 번째 토픽 : [('정병', 0.10089427612274049), ('장님', 0.09240659815241237), ('듣기', 0.0893809

In [ ]:
model.visualize_barchart()

## nr_topics="auto" 자동 토픽 축소 사용

In [ ]:
custom_tokenizer = CustomTokenizer(Okt)
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics="auto",
                 top_n_words=10,
                 calculate_probabilities=True)
topics, probs = model.fit_transform(documents)
for i in range(len(set(topics))):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('여자', 0.040012834918817256), ('남자', 0.033094556868798454), ('짱깨', 0.023652847965511193), ('사람', 0.02291282018511225), ('게이', 0.020556173410269392), ('전라도', 0.018590987788557965), ('원래', 0.01700048447521117), ('그게', 0.015005357237640845), ('수준', 0.01406752241028829), ('여자애', 0.013844864654277785)]
1 번째 토픽 : [('남자', 0.03850494499561311), ('여자', 0.03621703394457397), ('아이돌', 0.0237660262006295), ('연예인', 0.018920962536234), ('회사', 0.01888281759301704), ('사람', 0.018707838820423525), ('보기', 0.01707260978123147), ('보고', 0.016702278041725884), ('축구', 0.016517646571002676), ('거임', 0.015439109473263952)]
2 번째 토픽 : [('한국', 0.12669700994707847), ('조선족', 0.11096885761141198), ('중국', 0.10469821043320714), ('중국인', 0.0814657276281245), ('서울', 0.06554754794785943), ('한국인', 0.049976398319044636), ('북한', 0.046539056333938816), ('일본', 0.04041414166596587), ('인간', 0.03905653451794944), ('사람', 0.03557925057180429)]
3 번째 토픽 : [('정병', 0.07311419732023935), ('귀머거리', 0.06903293940816912), ('장님', 0.0

In [ ]:
model.visualize_barchart()

In [ ]:
model.visualize_topics()